In [1]:
!pip install imbDRL
!pip install --upgrade tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 35.5 MB/s 
     |████████████████████████████████| 6.7 MB 14.8 MB/s 
     |████████████████████████████████| 624 kB 70.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 18.3 MB 222 kB/s 
  Created wheel for gym: filename=gym-0.23.0-py3-none-any.whl size=697658 sha256=092d8a1b595ac2ec7e144e49dcf5db8c4b5c72b705029bd32151dc86e8e35f45
  Stored in directory: /root/.cache/pip/wheels/e7/2f/ab/68bf956c5dde73c1856d981e54292cf58385fb60bca10b7acd
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=9463ca7bf093a22abd18c60e41b5240f2a143fe204100d05629e0073ed35ee12
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built gym sklea

In [3]:
import csv
import os

from imbDRL.agents.ddqn import TrainDDQN
from imbDRL.data import get_train_test_val, load_csv
from imbDRL.metrics import classification_metrics, network_predictions
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from tqdm import tqdm
from numpy import load
import numpy as np
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
photos = load('./drive/MyDrive/AI/dogs_vs_cats_photos.npy')
labels = load('./drive/MyDrive/AI/dogs_vs_cats_labels.npy')
#photos = photos.reshape(1000, 200*200*3)
labels = labels.astype(int)
print(photos.shape, labels.shape)


(1000, 200, 200, 3) (1000,)


In [13]:
# load and confirm the shape
X_train, X_test, y_train, y_test = train_test_split(photos, labels, test_size=0.2, random_state=42, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  

layers = [Conv2D(32, (5, 5), padding="Same", activation="relu"),
          MaxPooling2D(pool_size=(2, 2)),
          Conv2D(32, (5, 5), padding="Same", activation="relu"),
          MaxPooling2D(pool_size=(2, 2)),
          Flatten(),
          Dense(256, activation="relu"),
          Dense(2, activation=None)]

episodes = 12_000  # Total episodes
warmup_steps = 5_000  # Amount of warmup steps to collect data with random policy
memory_length = 10_000  # Max length of the Replay Memory
collect_steps_per_episode = 1
target_update_period = 1_000
target_update_tau = 1
batch_size = 32
n_step_update = 4

learning_rate = 0.00025  # Learning rate
gamma = 0.1  # Discount factor
min_epsilon = 0.01  # Minimal and final chance of choosing random action
decay_episodes = 10_000  # Number of episodes to decay from 1.0 to `min_epsilon`

fp_dqn = "./results/dqn.csv"
fieldnames = ("Gmean", "F1", "Precision", "Recall", "TP", "TN", "FP", "FN")

# Create empty files
with open(fp_dqn, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

# Run the model ten times
for _ in tqdm(range(10)):
    # New train-test split

    model = TrainDDQN(episodes, warmup_steps, learning_rate, gamma, min_epsilon, decay_episodes, target_update_tau=target_update_tau,
                          collect_steps_per_episode=collect_steps_per_episode, target_update_period=target_update_period,
                          n_step_update=n_step_update, batch_size=batch_size, memory_length=memory_length, progressbar=False)

    model.compile_model(X_train, y_train, layers)
    model.train(X_val, y_val, "F1")

    # Predictions of model for `X_test`
    best_network = model.load_network(fp=model.model_path)
    y_pred = network_predictions(best_network, X_test)
    dqn_stats = classification_metrics(y_test, y_pred)

    # Write current DQN run to `fp_dqn`
    with open(fp_dqn, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writerow(dqn_stats)

  0%|          | 0/10 [00:00<?, ?it/s]WARNING:tensorflow:From /usr/local/lib/python3.8/dist-packages/tf_agents/replay_buffers/tf_uniform_replay_buffer.py:342: CounterV2 (from tensorflow.python.data.experimental.ops.counter) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
